In [1]:
from datetime import datetime
from fund.fund import Fund
from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio
from parameters.parameters import Parameters 
import pandas as pd
from returns.products import Products
from processor.processor import Processor as p
from time import sleep

In [2]:
portfolio = APortfolio(pricer_list.WEEKLY_STOCK_SPECULATION
                          ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolios = [portfolio]

In [3]:
backtest_start_date = datetime(2023,1,1)
backtest_end_date  = datetime.now()
current_start_date = datetime.now()

In [4]:
fund = Fund(portfolios,backtest_start_date,backtest_end_date,current_start_date)

In [5]:
fund.initialize_portfolios()
fund.initialize_backtesters()

In [6]:
fund.market.connect()
sp500 = fund.market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
fund.market.disconnect()

In [7]:
parameter = {
    "value":True,
    "ceiling":True,
    "classification":False,
    "rank":False,
    "short":False,
    "market_return":1.15
    ,"buy_day":2
    ,"sell_day":5
}

In [8]:
test_portfolio = fund.portfolios[0]
returns = test_portfolio.create_current_returns()
sim = test_portfolio.create_current_simulation()
merged = test_portfolio.merge_sim_returns(sim,returns)
merged = merged.sort_values(["year","week","day"]).dropna()

In [9]:
merged

,year,week,ticker,price_prediction,day,close,high,low,open,volume,...,adjvolume,divcash,splitfactor,quarter,prev_close,weekly_risk_return,bench_weekly_return,weekly_variance,market_weekly_cov,weekly_beta
977,2023,4,MMM,121.610181,4.0,115.25,115.7750,113.460,113.73,4114472.0,...,4114472.0,0.0,1.0,1.0,112.049339,-0.041206,-0.001115,0.000385,0.000046,0.119018
978,2023,4,AOS,58.627839,4.0,60.36,60.6400,58.820,58.90,1716813.0,...,1716813.0,0.0,1.0,1.0,58.299957,0.018342,-0.001115,0.000385,-0.000168,-0.437382
979,2023,4,ABT,112.179276,4.0,109.95,111.2600,109.810,110.65,4860837.0,...,4860837.0,0.0,1.0,1.0,110.525958,-0.006530,-0.001115,0.000385,-0.000023,-0.060733
980,2023,4,ABBV,150.011836,4.0,146.28,147.6799,146.015,147.31,4591481.0,...,4591481.0,0.0,1.0,1.0,146.071182,-0.008809,-0.001115,0.000385,-0.000196,-0.509751
981,2023,4,ACN,284.001107,4.0,277.27,279.4200,272.940,273.26,2101109.0,...,2101109.0,0.0,1.0,1.0,273.181752,0.005020,-0.001115,0.000385,0.000118,0.305766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47302,2023,24,YUM,132.211526,1.0,135.98,136.0600,134.440,134.44,1440819.0,...,1440819.0,0.0,1.0,2.0,134.630000,0.003428,0.014693,0.000094,0.000152,1.614815
47304,2023,24,ZBRA,272.210026,1.0,283.39,286.4400,280.610,283.52,367144.0,...,367144.0,0.0,1.0,2.0,279.990000,0.040430,0.014693,0.000094,0.000180,1.905922
47306,2023,24,ZBH,132.211526,1.0,136.83,137.1300,134.580,135.09,747714.0,...,747714.0,0.0,1.0,2.0,135.550000,0.038936,0.014693,0.000094,0.000223,2.364656
47308,2023,24,ZION,30.965639,1.0,29.77,30.3100,28.140,28.79,7691631.0,...,7691631.0,0.0,1.0,2.0,30.240000,0.052925,0.014693,0.000094,0.000169,1.788010


In [10]:
stuff = test_portfolio.recommendation(merged.copy(),parameter)

In [13]:
week = 24
positions = 20

In [19]:
rec.sort_values(["year","week"]).tail(40)

,year,week,ticker,weekly_delta,weekly_delta_sign,position,value,ceiling,classification,rank,short,market_return,buy_day,sell_day
18,2023,23,NVR,0.238269,1.0,0,True,True,False,False,False,1.15,2,5
38,2023,23,RL,0.065064,1.0,1,True,True,False,False,False,1.15,2,5
58,2023,23,SCHW,0.042483,1.0,2,True,True,False,False,False,1.15,2,5
78,2023,23,AMD,0.041002,1.0,3,True,True,False,False,False,1.15,2,5
98,2023,23,AMZN,0.038827,1.0,4,True,True,False,False,False,1.15,2,5
118,2023,23,BLK,0.038119,1.0,5,True,True,False,False,False,1.15,2,5
138,2023,23,MS,0.037166,1.0,6,True,True,False,False,False,1.15,2,5
158,2023,23,DAL,0.036116,1.0,7,True,True,False,False,False,1.15,2,5
178,2023,23,HPE,0.035895,1.0,8,True,True,False,False,False,1.15,2,5
198,2023,23,ALK,0.035515,1.0,9,True,True,False,False,False,1.15,2,5


# SET THE CURRENT WEEK

In [20]:
rec = stuff[0]
rec_filtered = rec[(rec["week"]==week)]
trades = rec_filtered.head(positions).merge(sp500[["ticker","Security","GICS Sector"]],on="ticker")
final = trades[["year","week","ticker","Security","GICS Sector","weekly_delta","weekly_delta_sign"]]

In [21]:
final

,year,week,ticker,Security,GICS Sector,weekly_delta,weekly_delta_sign
0,2023,24,AVGO,Broadcom Inc.,Information Technology,0.092040,1.0
1,2023,24,MPWR,Monolithic Power Systems,Information Technology,0.087785,1.0
2,2023,24,ADBE,Adobe Inc.,Information Technology,0.087476,1.0
3,2023,24,ORCL,Oracle Corporation,Information Technology,0.084792,1.0
4,2023,24,HPE,Hewlett Packard Enterprise,Information Technology,0.077548,1.0
5,2023,24,NXPI,NXP Semiconductors,Information Technology,0.069084,1.0
6,2023,24,NFLX,Netflix,Communication Services,0.068685,1.0
7,2023,24,TER,Teradyne,Information Technology,0.068299,1.0
8,2023,24,EPAM,EPAM Systems,Information Technology,0.067630,1.0
9,2023,24,JNPR,Juniper Networks,Information Technology,0.057411,1.0


In [ ]:
# test_portfolio.db.connect()
# test_portfolio.db.store("recs",final)
# test_portfolio.db.disconnect()